In [31]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf 
import numpy as np
from datetime import datetime
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)
from operator import attrgetter

In [3]:
df = pd.read_excel(r'C:\Users\kentj\Desktop\Coursera\Online Retail.xlsx')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [4]:
df.shape

(541909, 8)

In [5]:
df.info() # there are missing data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [9]:
df.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [8]:
df.dropna(subset = ['CustomerID'], inplace = True)

In [13]:
df.duplicated().sum() #check for duplicates

0

In [12]:
df = df.drop_duplicates()

In [14]:
df.describe() 

#Something is wrong here, we cant have negative quantity as well as 0 unitprice. 

,Quantity,UnitPrice,CustomerID
count,401604.000000,401604.000000,401604.000000
mean,12.183273,3.474064,15281.160818
std,250.283037,69.764035,1714.006089
min,-80995.000000,0.000000,12346.000000
25%,2.000000,1.250000,13939.000000
50%,5.000000,1.950000,15145.000000
75%,12.000000,3.750000,16784.000000
max,80995.000000,38970.000000,18287.000000


In [16]:
cond1 = df['Quantity'] >0 
cond2 = df['UnitPrice'] >0 

df = df[cond1 & cond2]
df.describe()

,Quantity,UnitPrice,CustomerID
count,392692.000000,392692.000000,392692.000000
mean,13.119702,3.125914,15287.843865
std,180.492832,22.241836,1713.539549
min,1.000000,0.001000,12346.000000
25%,2.000000,1.250000,13955.000000
50%,6.000000,1.950000,15150.000000
75%,12.000000,3.750000,16791.000000
max,80995.000000,8142.750000,18287.000000


In [21]:
df['invoicemonth'] = df['InvoiceDate'].dt.to_period('M')

In [218]:
group = df.groupby('CustomerID')['invoicemonth']
df['Cohort'] = group.transform('min')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,invoicemonth,Cohort,transaction_cost
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,2010-12,2010-12,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12,2010-12,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,2010-12,2010-12,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12,2010-12,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12,2010-12,20.34


In [204]:
#Active Customer in each cohort
cohort = df.groupby(['Cohort', 'invoicemonth']).agg(n_customers = ('CustomerID', 'nunique')).reset_index(drop = False)
cohort['period_number'] = (cohort.invoicemonth - cohort.Cohort).apply(attrgetter('n')) # we can also use this  
#cohort['period_number'] =  cohort['invoicemonth'].astype(int) - cohort['Cohort'].astype(int)

cohort_df = cohort.pivot_table(index = 'Cohort', columns = 'invoicemonth', values = 'n_customers')

In [205]:
#Customer retention %

cohort_df2 = cohort.pivot_table(index = 'Cohort', columns = 'period_number', values = 'n_customers')

cohort_size = cohort_df2.iloc[:,0]
retention_matrix = cohort_df2.divide(cohort_size, axis = 0)

cohort_size_df = pd.DataFrame(cohort_size).rename(columns = {0:'cohort_size'})

In [206]:
#Fill up zeros above diagonal and nan below diagonal

cohort_df = cohort_df.fillna(0)
m,n = cohort_df.shape
cohort_df[:] =np.where(np.arange(m)[:, None] >= np.arange(n)+1, np.nan, cohort_df)
cohort_df = pd.DataFrame(cohort_df)

df1 = cohort_df.sum()

df1= pd.DataFrame(df1)
df1['Delta'] = df1.diff()
df1.rename(columns={0: "Sub-Total"}, inplace = True)
df1 = df1.T
cohort_df = pd.concat([cohort_df, df1], axis = 0)
cohort_df.index.name = 'No. of Customer per month'
cohort_df = cohort_df.astype('Int64')

In [207]:
retention_matrix = retention_matrix.fillna(0)
m,n = retention_matrix.shape
retention_matrix[:] = np.where(np.arange(m)[:,None] >= np.arange(n)+(m-n+1),np.nan,np.fliplr(retention_matrix))
retention_matrix = pd.DataFrame(np.fliplr(retention_matrix))
retention_matrix = retention_matrix.rename(columns = lambda x: 'Month '+ str(x+1)) #Makes the column names to Month
retention_matrix = retention_matrix.rename(index = lambda x: 'Month '+ str(x+1))

df2 = pd.DataFrame(retention_matrix.mean())
df3 = pd.DataFrame(retention_matrix.median())

df2.rename(columns={0: "Average"}, inplace = True)
df3.rename(columns={0: "Median"}, inplace = True)

df4 = pd.concat([df2, df3], axis = 1)
df4 = df4.T

retention_matrix = pd.concat([retention_matrix, df4], axis = 0)
retention_matrix.index.name = 'Retention Rate Table'

In [140]:
retention_matrix

,Month 1,Month 2,Month 3,Month 4,Month 5,Month 6,Month 7,Month 8,Month 9,Month 10,Month 11,Month 12,Month 13
Retention Rate Table,,,,,,,,,,,,,
Month 1,1.0,0.366102,0.323164,0.384181,0.362712,0.397740,0.362712,0.349153,0.353672,0.395480,0.374011,0.502825,0.265537
Month 2,1.0,0.220624,0.266187,0.230216,0.321343,0.287770,0.247002,0.242206,0.299760,0.326139,0.364508,0.117506,NaN
Month 3,1.0,0.186842,0.186842,0.284211,0.271053,0.247368,0.252632,0.278947,0.247368,0.305263,0.068421,NaN,NaN
Month 4,1.0,0.150442,0.252212,0.199115,0.223451,0.168142,0.267699,0.230088,0.278761,0.086283,NaN,NaN,NaN
Month 5,1.0,0.213333,0.203333,0.210000,0.196667,0.226667,0.216667,0.260000,0.073333,NaN,NaN,NaN,NaN
Month 6,1.0,0.190141,0.172535,0.172535,0.207746,0.232394,0.264085,0.095070,NaN,NaN,NaN,NaN,NaN
Month 7,1.0,0.173554,0.157025,0.264463,0.231405,0.334711,0.095041,NaN,NaN,NaN,NaN,NaN,NaN
Month 8,1.0,0.180851,0.207447,0.223404,0.271277,0.111702,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Month 9,1.0,0.207101,0.248521,0.242604,0.124260,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Top 20% 
It is also a common operation to do a cohort analysis on the top 20% of paying customers. (High Value Paying Customer)

In [208]:
df['transaction_cost'] = df['Quantity'] * df['UnitPrice']

In [209]:
total_df = df.groupby('CustomerID')[['transaction_cost']].sum().sort_values(by = 'transaction_cost', ascending = False).reset_index(drop = False)
total_df['Cumulative Revenue'] = total_df['transaction_cost'].cumsum()
# there is a total of 4338 IDs here. We can take the to 20% of 4338, and run a cohort analysis on them. 
cutoffvalue = int(len(total_df['CustomerID']) * 0.2)
cutoffvalue #cutoffvalue have a number of 868 for this particular dataset. When added in the next few months, the value might change. 

867

In [220]:
total_df.tail() # the current cumulative revenue is $8,887,208.89

,CustomerID,transaction_cost,Cumulative Revenue
4333,16878.0,13.30,8887179.294
4334,17956.0,12.75,8887192.044
4335,16454.0,6.90,8887198.944
4336,14792.0,6.20,8887205.144
4337,16738.0,3.75,8887208.894


In [221]:
top_ = total_df.head(cutoffvalue) #the top 20% paying customer contributed a total revenue of $6,635,245.31, 
top_.tail() #the top 20% paying customer accounts to about 74.6% of the total revenue! 

,CustomerID,transaction_cost,Cumulative Revenue
862,12948.0,2064.95,6627004.211
863,12928.0,2062.70,6629066.911
864,16281.0,2062.06,6631128.971
865,14409.0,2059.45,6633188.421
866,12518.0,2056.89,6635245.311


In [210]:
top_df = df[df['CustomerID'].isin(list(top_['CustomerID'].unique()))]
top_df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,invoicemonth,Cohort,transaction_cost
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,2010-12,2010-12,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12,2010-12,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,2010-12,2010-12,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12,2010-12,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12,2010-12,20.34
...,...,...,...,...,...,...,...,...,...,...,...
541889,581585,22466,FAIRY TALE COTTAGE NIGHT LIGHT,12,2011-12-09 12:31:00,1.95,15804.0,United Kingdom,2011-12,2011-05,23.40
541890,581586,22061,LARGE CAKE STAND HANGING STRAWBERY,8,2011-12-09 12:49:00,2.95,13113.0,United Kingdom,2011-12,2010-12,23.60
541891,581586,23275,SET OF 3 HANGING OWLS OLLIE BEAK,24,2011-12-09 12:49:00,1.25,13113.0,United Kingdom,2011-12,2010-12,30.00
541892,581586,21217,RED RETROSPOT ROUND CAKE TINS,24,2011-12-09 12:49:00,8.95,13113.0,United Kingdom,2011-12,2010-12,214.80


In [211]:
#Active Customer in each cohort in top 20% 
top_cohort = top_df.groupby(['Cohort', 'invoicemonth']).agg(n_customers = ('CustomerID', 'nunique')).reset_index(drop = False)
top_cohort['period_number'] = (top_cohort.invoicemonth - top_cohort.Cohort).apply(attrgetter('n')) 

top_cohort_df = top_cohort.pivot_table(index = 'Cohort', columns = 'invoicemonth', values = 'n_customers')

In [216]:
top_cohort_df

invoicemonth,2010-12,2011-01,2011-02,2011-03,2011-04,2011-05,2011-06,2011-07,2011-08,2011-09,2011-10,2011-11,2011-12
No. of Customer per month,,,,,,,,,,,,,
2010-12,388,243,214,263,235,266,244,236,237,259,242,299,178
2011-01,<NA>,125,51,67,45,75,64,68,59,67,68,84,31
2011-02,<NA>,<NA>,80,31,36,46,42,34,45,51,38,46,15
2011-03,<NA>,<NA>,<NA>,98,38,55,43,48,47,61,46,58,18
2011-04,<NA>,<NA>,<NA>,<NA>,43,24,20,19,22,20,26,22,6
2011-05,<NA>,<NA>,<NA>,<NA>,<NA>,34,16,17,14,14,17,18,12
2011-06,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,29,8,13,19,16,21,8
2011-07,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,14,7,7,9,9,7
2011-08,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,10,5,7,6,3


In [212]:
#Customer retention % in top 20%

top_cohort_df2 = top_cohort.pivot_table(index = 'Cohort', columns = 'period_number', values = 'n_customers')

top_cohort_size = top_cohort_df2.iloc[:,0]
top_retention_matrix = top_cohort_df2.divide(top_cohort_size, axis = 0)

top_cohort_size_df = pd.DataFrame(top_cohort_size).rename(columns = {0:'cohort_size'})

In [213]:
#Fill up zeros above diagonal and nan below diagonal

top_cohort_df = top_cohort_df.fillna(0)
m,n = top_cohort_df.shape
top_cohort_df[:] =np.where(np.arange(m)[:, None] >= np.arange(n)+1, np.nan, top_cohort_df)
top_cohort_df = pd.DataFrame(top_cohort_df)

df5 = top_cohort_df.sum()

df5 = pd.DataFrame(df5)
df5['Delta'] = df5.diff()
df5.rename(columns={0: "Sub-Total"}, inplace = True)
df5 = df5.T
top_cohort_df = pd.concat([top_cohort_df, df5], axis = 0)
top_cohort_df.index.name = 'No. of Customer per month'
top_cohort_df = top_cohort_df.astype('Int64')

In [214]:
top_retention_matrix = top_retention_matrix.fillna(0)
m,n = top_retention_matrix.shape
top_retention_matrix[:] = np.where(np.arange(m)[:,None] >= np.arange(n)+(m-n+1),np.nan,np.fliplr(top_retention_matrix))
top_retention_matrix = pd.DataFrame(np.fliplr(top_retention_matrix))
top_retention_matrix = top_retention_matrix.rename(columns = lambda x: 'Month '+ str(x+1)) #Makes the column names to Month
top_retention_matrix = top_retention_matrix.rename(index = lambda x: 'Month '+ str(x+1))

df6 = pd.DataFrame(top_retention_matrix.mean())
df7 = pd.DataFrame(top_retention_matrix.median())

df6.rename(columns={0: "Average"}, inplace = True)
df7.rename(columns={0: "Median"}, inplace = True)

df8 = pd.concat([df6, df7], axis = 1)
df8 = df8.T

top_retention_matrix = pd.concat([top_retention_matrix, df8], axis = 0)
top_retention_matrix.index.name = 'Retention Rate Table'

In [217]:
writer = pd.ExcelWriter(r'C:\Users\kentj\Desktop\Coursera\Cohort_Analysis3.xlsx',engine='xlsxwriter')   
workbook=writer.book

worksheet=workbook.add_worksheet('Cohort - Analysis (ALL)')
writer.sheets['Cohort - Analysis (ALL)'] = worksheet

worksheet2=workbook.add_worksheet('Cohort - Analysis (Top 20%)')
writer.sheets['Cohort - Analysis (Top 20%)'] = worksheet2

colour_format = {'type': '3_color_scale', 'min_color': '#FFFFFF', 'mid_color': '#AFFFD3', 'max_color': '#00B050'}
bold = workbook.add_format({'bold': True, 'font_size':12 })#, 'font_name':'Tahoma'})

startrow1 = 3+1
endrow1 = startrow1 + cohort_df.shape[0]+1
worksheet.write_string(2, 0, 'Number of Customer Retained', bold)
cohort_df.to_excel(writer ,sheet_name='Cohort - Analysis (ALL)', startrow= startrow1 , startcol=0)
worksheet.conditional_format('B{}:AZ{}'.format(startrow1+1, endrow1-2), colour_format)

startrow2 = endrow1+6
endrow2 = startrow2 + retention_matrix.shape[0]+1
worksheet.write_string(startrow2-2, 0, '#Customer Retention Rate', bold)
retention_matrix.to_excel(writer ,sheet_name='Cohort - Analysis (ALL)',startrow= startrow2, startcol=0)
worksheet.conditional_format('B{}:AZ{}'.format(startrow2+2, endrow2-2), colour_format)

#Writing the Top 20% in the separate sheet 

startrow1 = 3+1
endrow1 = startrow1 + top_cohort_df.shape[0]+1
worksheet2.write_string(2, 0, 'Number of Customer in Top 20% Retained', bold)
top_cohort_df.to_excel(writer ,sheet_name='Cohort - Analysis (Top 20%)',startrow= startrow1, startcol=0)
worksheet2.conditional_format('B{}:AZ{}'.format(startrow1+1, endrow1-2), colour_format)

startrow2 = endrow1+6
endrow2 = startrow2 + top_retention_matrix.shape[0]+1
worksheet2.write_string(startrow2-2, 0, '#Customer Retention Rate in Top 20%', bold)
top_retention_matrix.to_excel(writer ,sheet_name='Cohort - Analysis (Top 20%)', startrow=startrow2, startcol=0)
worksheet2.conditional_format('B{}:AZ{}'.format(startrow2+2, endrow2-2), colour_format)

writer.save()